<a href="https://colab.research.google.com/github/gretelai/gretel-blueprints/blob/main/docs/notebooks/redact_pii.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Redact PII

In this blueprint, we will create a transform policy to identify and redact or replace PII with fake values. We will then use the SDK to transform a dataset and examine the results.


In [ ]:
%%capture

!pip install pyyaml Faker pandas
!pip install -U gretel-client

In [ ]:
# Specify your Gretel API key

import pandas as pd
from gretel_client import Gretel

pd.set_option("max_colwidth", None)

gretel = Gretel(api_key="prompt", project_name="redact-pii-transform")



In [6]:
from faker import Faker

# Use Faker to make training and test data.
def fake_pii_csv(filename, lines=100):
    fake = Faker()
    with open(filename, "w") as f:
        f.write("id,name,email,phone,visa,ssn,user_id\n")
        for i in range(lines):
            _name = fake.name()
            _email = fake.email()
            _phone = fake.phone_number()
            _cc = fake.credit_card_number()
            _ssn = fake.ssn()
            _id = f'user_{fake.numerify(text="#####")}'
            f.write(f"{i},{_name},{_email},{_phone},{_cc},{_ssn},{_id}\n")


fake_pii_csv("train.csv")
fake_pii_csv("test.csv")


In [ ]:
transform = gretel.submit_transform(
    config="transform/default",
    data_source="train.csv",
)


In [ ]:
from IPython.display import display

# Compare results.  Here is our "before."
input_df = pd.read_csv("test.csv")
print("test.csv head, before redaction")
display(input_df)

# And here is our "after."
print("test.csv head, after redaction")
transformed = transform.transformed_df
display(transformed)
